In [2]:
!git clone https://github.com/Mlawrence95/LinkedIn-Tech-Job-Data.git

Cloning into 'LinkedIn-Tech-Job-Data'...


In [4]:
%cd LinkedIn-Tech-Job-Data

/content/drive/MyDrive/kred/LinkedIn-Tech-Job-Data


In [13]:
!pip install -U pip setuptools wheel
!pip install spacy-transformers
#!pip install pytextrank
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 53.5/53.5 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 11.9/11.9 MB 8.5 MB/s eta 0:00:00
     -------------------------------------- 186.7/186.7 kB 5.7 MB/s eta 0:00:00
     ------------------------------------- 481.4/481.4 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
     ------------------------------------- 181.6/181.6 kB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB ? eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 7.2 MB/s eta 0:

In [1]:
#Employment type, Industries, Job function, Seniority level, company, company_id, context, date, description,
#education, location, months_experience, post_id, post_url, sal_high, sal_low, salary, title

# Full-time,Broadcast Media,Information Technology,Mid-Senior level,CyberCoders,21836.0,"{""@context"": ""http://schema.org"", ""@type"": ""JobPosting"", ""datePosted"": ""2021-07-09T17:39:28.000Z"", ""description"": ""Job Title: Senior Data Engineer&lt
import pandas as pd

df = pd.read_csv('datasets/SO-Indeed/jobs.csv')
records = df.to_dict(orient='records')

In [2]:
import spacy
import pywikibot
spacy.prefer_gpu()
from spacy import displacy
from collections import Counter
import en_core_web_sm, en_core_web_trf, en_core_web_md
nlp_trf = spacy.load("en_core_web_trf")

In [3]:
from spacy.pipeline.entity_linker import DEFAULT_NEL_MODEL
config = {
   "labels_discard": [],
   "n_sents": 0,
   "incl_prior": True,
   "incl_context": True,
   "model": DEFAULT_NEL_MODEL,
   "entity_vector_length": 64,
   "get_candidates": {'@misc': 'spacy.CandidateGenerator.v1'},
   "threshold": None,
}
#doc_trf = nlp_trf(records[0]['description'])
#nlp_trf.add_pipe('entity_linker', config=config)

In [4]:
from pywikibot.exceptions import NoPageError, InvalidTitleError

def get_wikidata(doc): #con trf e entity_linker
    graph = []
    # Iterate over the entities in the document
    for ent in doc.ents:
        # Get the entity label
        if ent.label_ != 'GPE' and ent.label_ != 'DATE':
            label = ent.label_
        else : continue
        # Get the entity text
        entity = ent.text

        # Search Wikidata for the entity
        site = pywikibot.Site("en", "wikipedia")
        page = pywikibot.Page(site, entity)

        # Get the Wikidata page for the entity
        try: graph.append({'entity': entity, "label": label, "WikidataId": page.data_item().id})
        except (NoPageError, InvalidTitleError) as e: pass
        #wd_page = page.data_item()
        # Print the graph
    #print(graph)
        # Add the entity to the graph
    return graph

#wikidata_ids = get_wikidata(doc_trf)

In [5]:
#print([d['WikidataId'].id  for d in wikidata_ids if 'Python' in d['entity']]) #wikidataID

In [ ]:
import csv
#entity_info_title : [{"Label": "Weight gain", "Type": "C", "WikidataId": "Q3403879", "Confidence": 1.0, "OccurrenceOffsets": [45], "SurfaceForms": ["Gain Weight"]}]
jobs_info = [ 'post_id', 'industries','job_function', 'title', 'abstract', 'post_url', 'entity_info_title', 'entity_info_abstract']

new_dict = []
djob = {}
with open('train_jobs.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    writer.writeheader()
    for job in records:
        djob["post_id"] = job['post_id']
        djob["industries"] = job['Industries']
        djob["job_function"] = job['Job function']
        djob["title"] = job['title']
        djob["abstract"] = job['description'].split('.')[0]
        djob["post_url"] = job['post_url']
        doc_trf = nlp_trf(job['description'])
        djob["entity_info_title"] = get_wikidata(doc_trf)
        djob["entity_info_abstract"] = []
        #print(djob)
        writer.writerow(djob)


Sleeping for 5.0 seconds, 2023-01-10 15:22:49
Sleeping for 5.0 seconds, 2023-01-10 15:23:27
Sleeping for 5.0 seconds, 2023-01-10 15:23:42
Sleeping for 5.0 seconds, 2023-01-10 15:24:10
Sleeping for 5.0 seconds, 2023-01-10 15:24:37
Sleeping for 5.0 seconds, 2023-01-10 15:38:56
Sleeping for 5.0 seconds, 2023-01-10 15:41:44
Sleeping for 5.0 seconds, 2023-01-10 15:42:21
Sleeping for 5.0 seconds, 2023-01-10 15:45:09
Sleeping for 5.0 seconds, 2023-01-10 15:46:00
Sleeping for 5.0 seconds, 2023-01-10 15:47:35


In [ ]:
with open('train_jobs.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    writer.writeheader()
    writer.writerows(new_dict)

#news | jobs
#id : post_id
#category: Industries
#subcategory : job function
#title: title
#abstract : requirements
#url : url
#entity_info_title : only one [{label, type, wikidataId, confidence, occurrence offsets, surface forms}]
#entity_info_abstract: all the first 3[{label, type, wikidataId, confidence, occurrence offsets, surface forms}]